In [152]:
# %load solution.py
# Import important libraries
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
from itertools import chain
from itertools import repeat
from collections import OrderedDict

import xml.etree.ElementTree as ET

config = {}

# E.g. "1.256660 0.431805 -4.981400"
def parse_coords(text):
    return [float(x) for x in text.split(' ')]

def iter_dataset(xml_tree):
    for child in xml_tree.getroot():
        name = int(child.tag.split('_')[1])
        try:
            energy =  float(child.find('energy').text)
        except AttributeError:
            energy = np.nan
        atoms = [parse_coords(element.text) for element in child.find('coordinates').findall('c')]
        for i, coords in enumerate(atoms):
            yield {'Entry':name, 'Energy':energy, 'Atom': i, 'X':coords[0], 'Y':coords[1], 'Z':coords[2]}

def parse_dataset(xml_file):
    xml_tree = ET.parse(xml_file)
    training_set = list(iter_dataset(xml_tree))

    return pd.DataFrame(training_set, columns=('Entry', 'Energy', 'Atom', 'X', 'Y', 'Z'))
    
def get_pos(data, entry):
    # Convert the X, Y, Z position for entry to a numpy array of size 60x3
    
    # Get single entry
    E = data[data['Entry'] == entry]
    if E.empty:
        print('Invalid Entry id!')
        return None
    
    # Get the position in format Nx3
    E_ = E.apply(lambda row: [row['X'], row['Y'], row['Z']], axis=1).values
    
    # Transform it to a numpy array
    Epos = np.reshape(list(chain(*E_)), (60, 3))
    
    return Epos


def get_distance(pos0, pos1, method='atom_pos'):
    # Calculate a distance value between e0 and e1 based on
    # method='atom_pos' ... their cummulative difference in atom positions
    # method='mesh_size' ... the abs. diff in mean atom gap size (i.e mesh size)
    # method='mesh_size_variance' ... the abs. diff of variance of the mean atom gap size (i.e variance of the mesh size)
    
    if method == 'atom_pos':
        # Calculate the distance matrix
        D = cdist(pos0, pos1, metric='euclidean')

        # Find the closest match for each point
        assignment = np.argsort(D, axis=1)[:, 0]

        # Calculate distance between each point to its assigned point
        distance = np.sum(np.sqrt(np.sum((pos0 - pos1[assignment, :])**2, axis=1)))
        
    elif method == 'mesh_size':
        # For each atom calculate the mean distance to its three closest neighbours
        D0 = cdist(pos0, pos0, metric='euclidean')
        D0.sort(axis=1)
        D0_mesh_size = np.mean(D0[:, 1:4])

        D1 = cdist(pos1, pos1, metric='euclidean')
        D1.sort(axis=1)
        D1_mesh_size = np.mean(D1[:, 1:4])
        
        distance = np.abs(D0_mesh_size - D1_mesh_size)

    elif method == 'mesh_size_variance':
        # For each atom calculate the mean distance to its three closest neighbours
        D0 = cdist(pos0, pos0, metric='euclidean')
        D0.sort(axis=1)
        D0_mesh_size_var = np.var(np.mean(D0[:, 1:4], axis=1))

        D1 = cdist(pos1, pos1, metric='euclidean')
        D1.sort(axis=1)
        D1_mesh_size_var = np.var(np.mean(D1[:, 1:4], axis=1))
        
        distance = np.abs(D0_mesh_size_var - D1_mesh_size_var)
       
    return distance


def calculate_ranking(prediction_data, lookup_data, distance_method = ''):
    # For each entry in 'prediction_data' rank all entries in 'data'
    #
    # Return a ordered Dictionary containg for each prediction_data Entry
    # a tuple describing the similary/distance to each entry in the lookup table.
    
    prediction_entries = prediction_data['Entry'].drop_duplicates()
    lookup_entries = lookup_data['Entry'].drop_duplicates()
    
    results = OrderedDict()
    for pre in prediction_entries:
        ranking = []
        e0pos = get_pos(prediction_data, pre)
        for (e0, e1) in zip(repeat(pre), lookup_entries):
            e1pos = get_pos(lookup_data, e1)
            d = get_distance(e1pos, e0pos, method=distance_method)
            ranking.append((d, e1))

        ranking.sort()
        results[pre] = ranking
    
    return results


def get_predictions(results, lookup_data):
    # Based on the ranking calculate a energy value for each entry by
    # taking the mean energy value of its 3 closest matches.

    entries = []
    predictions = []
    for entry_id in results.keys():
        entries.append(entry_id)
        closest_entries = [res[1] for res in results[entry_id][0:3]]
        predictions.append(np.mean(get_energies(lookup_data, closest_entries)))
    
    return entries, predictions


def single_stage_prediction(training, validation):
    ranking = calculate_ranking(validation, training, distance_method='atom_pos')   
    entries, predictions = get_predictions(ranking, training)
    return entries, predictions


def two_stage_prediction(training, validation, energy_sw=0.1):
    ranking = calculate_ranking(validation, training, distance_method='atom_pos')   
    entries, predictions = get_predictions(ranking, training)

    # For each entry in the first prediction generate a subset of the training data
    # and apply another distance metric to the subset in order to calculate
    # a improved prediction
    new_predictions = []
    for entry_id, predicted_energy in zip(entries, predictions):
        # Calculate a subset of the data
        training_subset = training[(training['Energy'] > (predicted_energy-energy_sw)) & (training['Energy'] < (predicted_energy+energy_sw))]
        validation_subset = validation[validation['Entry'] == entry_id]

        new_ranking = calculate_ranking(validation_subset, training_subset, distance_method='mesh_size_variance')   
        _, new_prediction = get_predictions(new_ranking, training_subset)
    
        new_predictions.append(new_prediction[0])
    
    return entries, new_predictions


############### HELPER FUNCTIONS - NOT PART OF THE ALGORITHM ###############

def evaluate_prediction(entry_ids, predicted_energies, lookup_table):
    # Calculate the prediction error
    prediction_errors = []
    for entry_id, predicted_energy in zip(entry_ids, predicted_energies):
        real_energy = lookup_table[lookup_table['Entry'] == entry_id]['Energy'].values[0]
        prediction_errors.append(predicted_energy - real_energy)
        
    return np.array(prediction_errors)


def cross_validation(n_tests, n_entries, training_data, prediction_function, kwargs={}):
    prediction_errors = np.zeros(shape=(n_tests, n_entries))    
    for n in range(0, n_tests):
        # Split the training data into a new set of training and validation data in order to test the algorithm
        validation_entries = set(np.random.choice(training_data['Entry'].unique(), n_entries, replace=False))
        training_entries = set(training_data['Entry'].unique()) - validation_entries
        
        print('Running Test (%d/%d) with validation entries %s ...' % (n+1, n_tests, validation_entries))
        
        training = training_data[training_data['Entry'].isin(training_entries)]
        validation = training_data[training_data['Entry'].isin(validation_entries)]

        entries, predictions = prediction_function(training, validation, **kwargs)
        
        prediction_errors[n, :] = evaluate_prediction(entries, predictions, training_data)
    
    return prediction_errors


def get_energies(table, entries):
    return [table[table['Entry'] == entry]['Energy'].values[0] for entry in entries]
        
def get_closest_entries(table, energy):
    uT = table[['Entry', 'Energy']].drop_duplicates()    
    energies = uT['Energy'].values
    entries = uT['Entry'].values    
        
    diff_energies = (energies - energy)**2
    closest_energies = np.argsort(diff_energies)
    closest_entries = entries[closest_energies]
    
    return closest_entries, energies[closest_energies]


In [11]:
print('Loading data ...')

# Real data   
training_data = parse_dataset('data/new_training_set.xml')
#validation = parse_dataset('data/new_validation_set.xml')

n_entries = 5
validation_entries = set(np.random.choice(training_data['Entry'].unique(), n_entries, replace=False))
training_entries = set(training_data['Entry'].unique()) - validation_entries

training = training_data[training_data['Entry'].isin(training_entries)]
validation = training_data[training_data['Entry'].isin(validation_entries)]



Loading data ...


# First stage lookup and prediction (traditional)

In [122]:
ranking = calculate_ranking(validation, training, distance_method='atom_pos')   
entries, predictions = get_predictions(ranking, training)

new_predictions = []
for entry_id, predicted_energy in zip(entries, predictions):
    # Calculate a subset of the data
    training_subset = training[(training['Energy'] > (predicted_energy-0.1)) & (training['Energy'] < (predicted_energy+0.1))]
    validation_subset = validation[validation['Entry'] == entry_id]

    new_ranking = calculate_ranking(validation_subset, training_subset, distance_method='mesh_size')   
    _, new_prediction = get_predictions(new_ranking, training_subset)
    
    new_predictions.append(new_prediction[0])

In [110]:
ranking = calculate_ranking(validation, training, distance_method='atom_pos')   
entries, predictions = get_predictions(ranking, training)



In [123]:
entries, new_predictions

([3, 17, 71, 148, 199],
 [-0.22383333333333333,
  -0.061199999999999997,
  -0.07173333333333333,
  0.0041999999999999997,
  -0.088166666666666671])

In [161]:
print('Cross validate two stage prediction ...')
training = parse_dataset('data/new_training_set.xml')
cum_errors = cross_validation(10, 5, training, prediction_function=two_stage_prediction, kwargs={'energy_sw': 0.05})
print('Mean prediction error %3.3f, Var %3.3f, abs.sum %3.3f' % (cum_errors.mean(), cum_errors.var(), np.sum(np.abs(cum_errors))))

Cross validate two stage prediction ...
Running Test (1/10) with validation entries {144, 70, 173, 46, 89} ...
Running Test (2/10) with validation entries {89, 203, 148, 142, 33} ...
Running Test (3/10) with validation entries {200, 41, 130, 46, 87} ...
Running Test (4/10) with validation entries {65, 94, 5, 133, 174} ...
Running Test (5/10) with validation entries {180, 3, 131, 182, 15} ...
Running Test (6/10) with validation entries {0, 113, 218, 51, 152} ...
Running Test (7/10) with validation entries {128, 113, 88, 38, 103} ...
Running Test (8/10) with validation entries {128, 48, 31, 54, 47} ...
Running Test (9/10) with validation entries {129, 30, 69, 133, 15} ...
Running Test (10/10) with validation entries {216, 139, 76, 102, 135} ...
Mean prediction error 0.013, Var 0.012, abs.sum 4.022


# Cross validate two stage prediction

In [132]:
print('Cross validate single stage prediction ...')
training = parse_dataset('data/new_training_set.xml')
cum_errors = cross_validation(50, 5, training, prediction_function=single_stage_prediction)
print('Mean prediction error %3.3f, Var %3.3f, abs.sum %3.3f' % (cum_errors.mean(), cum_errors.var(), np.sum(np.abs(cum_errors))))

Cross validate single stage prediction ...
Running Test (1/50) with validation entries {54, 171, 116, 86, 103} ...
Running Test (2/50) with validation entries {96, 1, 45, 71, 63} ...
Running Test (3/50) with validation entries {120, 59, 123, 140, 183} ...
Running Test (4/50) with validation entries {129, 164, 27, 92, 85} ...
Running Test (5/50) with validation entries {88, 151, 138, 156, 31} ...
Running Test (6/50) with validation entries {113, 194, 12, 122, 183} ...
Running Test (7/50) with validation entries {57, 18, 91, 54, 14} ...
Running Test (8/50) with validation entries {48, 57, 147, 209, 155} ...
Running Test (9/50) with validation entries {19, 57, 122, 67, 119} ...
Running Test (10/50) with validation entries {26, 78, 34, 83, 118} ...
Running Test (11/50) with validation entries {130, 35, 124, 21, 207} ...
Running Test (12/50) with validation entries {43, 83, 27, 60, 29} ...
Running Test (13/50) with validation entries {120, 113, 163, 21, 166} ...
Running Test (14/50) with va

In [149]:
print('Cross validate two stage prediction ...')
training = parse_dataset('data/new_training_set.xml')
cum_errors = cross_validation(50, 5, training, prediction_function=two_stage_prediction, kwargs={'energy_sw': 0.1})
print('Mean prediction error %3.3f, Var %3.3f, abs.sum %3.3f' % (cum_errors.mean(), cum_errors.var(), np.sum(np.abs(cum_errors))))

Cross validate two stage prediction ...
Running Test (1/50) with validation entries {64, 80, 20, 197, 118} ...
Running Test (2/50) with validation entries {88, 73, 45, 97, 160} ...
Running Test (3/50) with validation entries {56, 194, 42, 123, 159} ...
Running Test (4/50) with validation entries {128, 105, 43, 174, 183} ...
Running Test (5/50) with validation entries {136, 82, 91, 112, 95} ...
Running Test (6/50) with validation entries {153, 154, 164, 4, 215} ...
Running Test (7/50) with validation entries {135, 193, 156, 104, 199} ...
Running Test (8/50) with validation entries {24, 217, 133, 22, 183} ...
Running Test (9/50) with validation entries {72, 81, 43, 179, 17} ...
Running Test (10/50) with validation entries {144, 50, 115, 179, 214} ...
Running Test (11/50) with validation entries {8, 129, 218, 204, 4} ...
Running Test (12/50) with validation entries {217, 114, 203, 93, 215} ...
Running Test (13/50) with validation entries {0, 162, 107, 94, 119} ...
Running Test (14/50) wit

In [147]:
print('Cross validate two stage prediction ...')
training = parse_dataset('data/new_training_set.xml')
cum_errors = cross_validation(50, 5, training, prediction_function=two_stage_prediction, kwargs={'energy_sw': 0.05})
print('Mean prediction error %3.3f, Var %3.3f, abs.sum %3.3f' % (cum_errors.mean(), cum_errors.var(), np.sum(np.abs(cum_errors))))

Cross validate two stage prediction ...
Running Test (1/50) with validation entries {73, 177, 147, 214, 30} ...
Running Test (2/50) with validation entries {112, 211, 125, 87, 111} ...
Running Test (3/50) with validation entries {152, 100, 147, 4, 103} ...
Running Test (4/50) with validation entries {209, 203, 97, 14, 159} ...
Running Test (5/50) with validation entries {137, 154, 41, 118, 207} ...
Running Test (6/50) with validation entries {89, 19, 36, 46, 127} ...
Running Test (7/50) with validation entries {116, 162, 11, 20, 102} ...
Running Test (8/50) with validation entries {113, 218, 147, 193, 154} ...
Running Test (9/50) with validation entries {208, 210, 48, 213, 111} ...
Running Test (10/50) with validation entries {72, 163, 112, 117, 39} ...
Running Test (11/50) with validation entries {137, 194, 35, 140, 94} ...
Running Test (12/50) with validation entries {26, 211, 140, 190, 154} ...
Running Test (13/50) with validation entries {41, 51, 36, 209, 25} ...
Running Test (14/5

In [163]:
print('Try with atom_pos and mesh_size_variance')
print('Cross validate two stage prediction ...')
training = parse_dataset('data/new_training_set.xml')
cum_errors = cross_validation(50, 5, training, prediction_function=two_stage_prediction, kwargs={'energy_sw': 0.05})
print('Mean prediction error %3.3f, Var %3.3f, abs.sum %3.3f' % (cum_errors.mean(), cum_errors.var(), np.sum(np.abs(cum_errors))))

Try with atom_pos and mesh_size_variance
Cross validate two stage prediction ...
Running Test (1/50) with validation entries {192, 148, 5, 46, 111} ...
Running Test (2/50) with validation entries {144, 154, 203, 85, 215} ...
Running Test (3/50) with validation entries {193, 130, 215, 97, 41} ...
Running Test (4/50) with validation entries {32, 121, 46, 113, 143} ...
Running Test (5/50) with validation entries {116, 143, 140, 69, 199} ...
Running Test (6/50) with validation entries {48, 103, 111, 197, 183} ...
Running Test (7/50) with validation entries {201, 18, 36, 213, 199} ...
Running Test (8/50) with validation entries {153, 42, 31, 62, 151} ...
Running Test (9/50) with validation entries {48, 76, 109, 62, 127} ...
Running Test (10/50) with validation entries {9, 180, 111, 22, 105} ...
Running Test (11/50) with validation entries {112, 56, 138, 131, 82} ...
Running Test (12/50) with validation entries {217, 172, 109, 182, 86} ...
Running Test (13/50) with validation entries {0, 37,

In [99]:
e = 3
test_entry = entries[e]
real_energy = validation[validation['Entry'] == test_entry]['Energy'].values[0]
predicted_energy = predictions[e]
print('For Entry %d , real energy %3.3f' % (test_entry, real_energy))
print('Prediction for entry %d: %3.3f' % (test_entry, predicted_energy))
ranking_closest = ranking[test_entry][0:5]
ranking_entries = [r[2] for r in ranking_closest]
print('Closest_ranking entries: %s' % (ranking_entries))

optimal_closest, optimal_energies = get_closest_entries(training, real_energy)
print('Optimal selection %s, %s, mean %3.3f' % (optimal_closest[0:5], optimal_energies[0:5], np.mean(optimal_energies[0:5])))

# Generate a subset of the entries in trianing based on the prediction
utraining = training[['Entry', 'Energy']].drop_duplicates()
subset = utraining[(utraining['Energy'] > (predicted_energy-0.1)) & (utraining['Energy'] < (predicted_energy+0.1))]
subset_entries = subset['Entry'].values

print('Subset based on prediction\n%s' %(subset_entries))

For Entry 148 , real energy 0.003
Prediction for entry 148: 0.011
Closest_ranking entries: [150, 149, 147, 151, 153]
Optimal selection [216 138 121 106 107], [ 0.0024  0.0024  0.0035  0.0011  0.0048], mean 0.003
Subset based on prediction
[ 12  13  14  15  21  26  48  50  51  52  66  67  69  70  80  83 103 105
 106 107 108 121 122 123 124 125 136 137 138 139 140 147 149 150 151 152
 153 154 160 161 162 163 164 182 186 200 210 211 216]


In [40]:
subset_entries

array([  0,  12,  13,  14,  15,  21,  24,  25,  26,  27,  32,  48,  50,
        51,  52,  55,  67,  69,  70,  78,  80,  82,  83, 103, 104, 105,
       106, 107, 108, 110, 119, 121, 122, 123, 124, 125, 128, 134, 136,
       137, 138, 139, 140, 143, 144, 147, 149, 150, 151, 152, 153, 154,
       160, 161, 162, 163, 164, 171, 172, 175, 180, 182, 185, 186, 200,
       210, 211, 212, 216])

In [69]:
def characterise_entry(data_table, entry_ids):
    all_characters = []
    for entry_id in entry_ids:
        characters = {}
        pos = get_pos(data_table, entry_id)
        
        # Calculate the distance matrix
        D = cdist(pos, pos, metric='euclidean')
        sD = D.copy()
        sD.sort(axis=1)
        characters['mesh_distance'] = np.mean(sD[:, 1:4])

        all_characters.append(characters)
    return all_characters

In [101]:
test_char = characterise_entry(validation, [test_entry])
rest_char = characterise_entry(training, subset_entries)

print('Test Entry %s' % (test_char))
print('Others\n%s' % rest_char)

Test Entry [{'mesh_distance': 2.7128387642113903}]
Others
[{'mesh_distance': 2.7117290311915774}, {'mesh_distance': 2.7119606322721932}, {'mesh_distance': 2.7078978321662799}, {'mesh_distance': 2.7072373478151017}, {'mesh_distance': 2.6856590599362495}, {'mesh_distance': 2.6871674149923326}, {'mesh_distance': 2.6817326360900502}, {'mesh_distance': 2.7105697782581513}, {'mesh_distance': 2.7048556868771665}, {'mesh_distance': 2.6922581244628501}, {'mesh_distance': 2.6773050147782076}, {'mesh_distance': 2.7138005124619609}, {'mesh_distance': 2.710730224083064}, {'mesh_distance': 2.6805387605221407}, {'mesh_distance': 2.6696537246531111}, {'mesh_distance': 2.6884313049189208}, {'mesh_distance': 2.688479609839082}, {'mesh_distance': 2.7126036022794886}, {'mesh_distance': 2.7139531634814231}, {'mesh_distance': 2.7134423896835314}, {'mesh_distance': 2.7038260299696772}, {'mesh_distance': 2.712251432641402}, {'mesh_distance': 2.7099816414647266}, {'mesh_distance': 2.7084723402079698}, {'mesh_d

In [160]:
pos = get_pos(training, 0)
D = cdist(pos, pos, metric='euclidean')
sD = D.copy()
sD.sort(axis=1)

array([ 4.25556265,  2.04974858,  2.54751913,  2.49542602,  3.99036079,
        4.35478608,  3.58938472,  4.78022724,  3.55014216,  4.3378977 ,
        4.40292995,  5.19201513,  2.79106657,  3.78204809,  5.64012917,
        4.79728601,  4.95482542,  3.59118626,  5.75552764,  4.43885973,
        6.31293727,  5.7700784 ,  6.271003  ,  4.47857438,  4.51784364,
        3.6139715 ,  4.07431902,  5.17409053,  4.51534389,  4.30386145,
        4.90205037,  5.87167186,  6.72958229,  6.31079537,  5.52663515,
        5.63885774,  5.90172222,  6.93463571,  6.42064926,  5.94772687,
        4.39828149,  7.00167225,  6.74585549,  7.23314105,  4.84333516,
        4.51691913,  6.72191267,  6.26962099,  6.75865645,  4.99732055,
        5.79332566,  7.28647164,  6.67763817,  4.3346693 ,  5.4211885 ,
        6.37946002,  4.79224952,  5.15842198,  6.39650756,  5.74993117])

In [102]:
mesh_distance = [c['mesh_distance'] for c in rest_char]
closest = subset_entries[np.argsort(mesh_distance)]

np.mean(get_energies(training, closest[0:3]))

9.9999999999998243e-05